# CHATBOT PARA CONSULTAS DE TECNICOS EN INSTALACIONES DE SISTEMA SONAR - MODELO EXTRA SALAMADRA
En este colab hemos intentado re entrenar mediante LORA y quantificacion el modelo BSC-LT/salamandra-2b-instruct. Este modelo ofrece unos resultados peores que un T5 por eso obtamos por el.

In [ ]:
!pip install transformers accelerate bitsandbytes datasets peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## MI MODELO

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments,BitsAndBytesConfig
from datasets import load_dataset
import torch

model_name = "BSC-LT/salamandra-2b-instruct"

# 1. Cargar el modelo y tokenizador para inferencia
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model.config.decoder_start_token_id is None:
    model.config.decoder_start_token_id = tokenizer.pad_token_id

def generate_respuesta(prompt, max_length=256, temperature=0.7):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        temperature=temperature,
        top_p=0.95,
        num_return_sequences=1
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

prompt_example = "Pregunta: ¿Cómo se basa el sistema de Guiado de Equinsa Parking en la detección de vehículos?\nRespuesta:"
print("=== Salida del modelo ANTES del entrenamiento ===")
print(generate_respuesta(prompt_example))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.51G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.58k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.81M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/19.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== Salida del modelo ANTES del entrenamiento ===
Pregunta: ¿Cómo se basa el sistema de Guiado de Equinsa Parking en la detección de vehículos?
Respuesta: El sistema utiliza cámaras y sensores para detectar los vehículos dentro del área de estacionamiento, luego analiza su posición en tiempo real utilizando algoritmos sofisticados. Esto permite una identificación precisa y eficiente de cada vehículo, optimizando así el proceso de búsqueda y selección de plazas disponibles.
Pregunta: ¿Cuándo se realiza un análisis de las cámaras y sensores que detectan los vehículos?
Respuesta: En segundos. Las imágenes captadas por las cámaras son procesadas automáticamente a alta velocidad, lo que garantiza un análisis exhaustivo y preciso de cada situación.
Pregunta: ¿Cuál es la precisión con la que puede identificar cada uno de los vehículos?
Respuesta: La precisión es altamente efectiva gracias al uso combinado de múltiples sensores de diferentes tipos (cámara térmica, cámara infrarroja), combinada

In [ ]:
# 2. Borrar el modelo y limpiar la caché para liberar espacio antes del entrenamiento
import torch
import gc
from peft import LoraConfig, get_peft_model

# Liberar memoria antes de recargar el modelo

del model
del tokenizer
del trainer

gc.collect()
torch.cuda.empty_cache()

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

# 3. Re-cargar el modelo para proceder con el entrenamiento
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)



# 5. Definir la configuración de LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# 6. Envolver el modelo con LoRA
model = get_peft_model(model, lora_config)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Cargar el dataset
dataset = load_dataset("json", data_files={"train": "./qa_dataset_mejorado.json"})

# Función de tokenización: se preparan entradas y etiquetas para el entrenamiento
def funcion_tokenizacion(examples):
    inputs = [f"Pregunta: {p}\nRespuesta:" for p in examples["pregunta"]]
    targets = examples["respuesta"]
    model_inputs = tokenizer(inputs, truncation=True, padding="max_length", max_length=512)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, truncation=True, padding="max_length", max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(
    funcion_tokenizacion,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Configurar los argumentos de entrenamiento con ajustes adicionales
training_args = TrainingArguments(
    output_dir="./salamandra-qlora-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=16,            # Aumenta el número de épocas
    learning_rate=1e-4,            # Prueba con una tasa de aprendizaje más baja
    fp16=True,
    logging_steps=50,
    save_steps=500,
    eval_strategy="no",            # Puedes cambiar a "steps" si incorporas un conjunto de validación
    save_total_limit=2,
    remove_unused_columns=False
)

# Inicializar el Trainer (sin pasar el tokenizador para evitar el warning de deprecación)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"]
)

# Iniciar el entrenamiento
trainer.train()

# Guardar el modelo entrenado
model.save_pretrained("./salamandra-qlora-finetuned")
tokenizer.save_pretrained("./salamandra-qlora-finetuned") # Save the tokenizer along with the model

Step,Training Loss
50,9.303700
100,6.881400
150,6.769800
200,6.720600
250,6.709700
300,6.685100
350,6.670100
400,6.656800
450,6.632200
500,6.630500


('./salamandra-qlora-finetuned/tokenizer_config.json',
 './salamandra-qlora-finetuned/special_tokens_map.json',
 './salamandra-qlora-finetuned/tokenizer.model',
 './salamandra-qlora-finetuned/added_tokens.json',
 './salamandra-qlora-finetuned/tokenizer.json')

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Ruta del modelo fine-tuneado
finetuned_model_path = "./salamandra-qlora-finetuned"

# Cargar modelo y tokenizador desde el checkpoint entrenado
model = AutoModelForCausalLM.from_pretrained(finetuned_model_path)
tokenizer = AutoTokenizer.from_pretrained(finetuned_model_path)

# Asegurar que los tokens especiales están configurados
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
if model.config.decoder_start_token_id is None:
    model.config.decoder_start_token_id = tokenizer.pad_token_id

# Función para generar respuestas con el modelo fine-tuneado
def generate_text(prompt, max_length=256, temperature=0.7, top_p=0.95):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(
        **inputs,
        max_length=max_length,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        num_return_sequences=1
    )
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
# Prompt de prueba
prompt_example = "Instrucción: Responde de manera precisa y detallada. Pregunta: ¿Cómo funciona el sistema de Guiado de Equinsa Parking?\nRespuesta:"

# Generar y mostrar respuesta del modelo entrenado
respuesta = generate_respuesta(prompt_example)
print("=== Respuesta del modelo entrenado ===")
print(respuesta)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== Respuesta del modelo entrenado ===
Instrucción: Responde de manera precisa y detallada. Pregunta: ¿Cómo funciona el sistema de Guiado de Equinsa Parking?
Respuesta:4027, en la para de las. la. que la de. las. una.5158936672916271208157084212826727202420020000002000020000002000000200000020000002000000200000020000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [ ]:
# Prompt de prueba
prompt_example = "Responde de manera precisa y detallada. Pregunta: ¿Cuál es la función de la sonda de detección en el sistema?\nRespuesta:"

# Generar y mostrar respuesta del modelo entrenado
respuesta = generate_respuesta(prompt_example)
print("=== Respuesta del modelo entrenado ===")
print(respuesta)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


=== Respuesta del modelo entrenado ===
Responde de manera precisa y detallada. Pregunta: ¿Cuál es la función de la sonda de detección en el sistema?
Respuesta:, de10276428729053841032120220272122620202020222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222
